In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, f_oneway, ttest_ind
from pathlib import Path

# Significance level for tests
alpha = 0.05


Attempting to load data from: /home/rahel/Desktop/ AlphaCare Insurance Solutions/data/MachineLearningRating_v3.txt


FileNotFoundError: Could not load dataset: The 'low_memory' option is not supported with the 'python' engine